# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [130]:
import pandas as pd

from patsy import dmatrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import numpy as np

# Carregar os dados
data = pd.read_csv('previsao_de_renda.csv')

# Remover colunas desnecessárias
data = data.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1)

# Converter variáveis categóricas em dummies
data = pd.get_dummies(data)

# Separar variáveis independentes (X) e dependente (y)
X = data.drop('renda', axis=1)
y = data['renda']

# Dividir os dados em conjuntos de treinamento e teste (75% treinamento, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Lidar com valores ausentes apenas nos dados de treinamento
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Escalonar os recursos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Definir os valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Para armazenar os resultados
ridge_results = {}

# Iterar sobre os valores de alpha
for alpha in alphas:
    # Inicializar o modelo de regressão Ridge
    ridge = Ridge(alpha=alpha)
    
    # Ajustar o modelo aos dados de treinamento
    ridge.fit(X_train_scaled, y_train)
    
    # Fazer previsões nos dados de teste
    y_pred = ridge.predict(X_test_scaled)
    
    # Calcular o R^2 nos dados de teste
    r2 = r2_score(y_test, y_pred)
    
    # Armazenar os resultados
    ridge_results[alpha] = r2

# Imprimir os resultados
for alpha, r2 in ridge_results.items():
    print(f"Alpha = {alpha}: R^2 = {r2}")

Alpha = 0: R^2 = 0.2657497438376205
Alpha = 0.001: R^2 = 0.2690308236549174
Alpha = 0.005: R^2 = 0.2690308213219027
Alpha = 0.01: R^2 = 0.26903081840093557
Alpha = 0.05: R^2 = 0.2690307948459575
Alpha = 0.1: R^2 = 0.26903076493752254


O melhor modelo seria aquele com o maior valor de R^2 nos dados de teste, pois indica que o modelo tem a melhor capacidade de explicar a variabilidade da variável dependente (renda). Neste caso, o R^2 é praticamente o mesmo para todos os valores de alpha, o que sugere que a regularização Ridge não está tendo um efeito significativo na performance do modelo. No entanto, o modelo com alpha igual a 0.001 tem um R^2 ligeiramente maior, então poderíamos considerá-lo como o "melhor" modelo entre os testados.

In [131]:
from sklearn.linear_model import Lasso

# Definir os valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Para armazenar os resultados
lasso_results = {}

# Iterar sobre os valores de alpha
for alpha in alphas:
    # Inicializar o modelo de regressão Lasso
    lasso = Lasso(alpha=alpha)
    
    # Ajustar o modelo aos dados de treinamento
    lasso.fit(X_train_scaled, y_train)
    
    # Calcular o R^2 nos dados de treinamento
    r2_train = lasso.score(X_train_scaled, y_train)
    
    # Calcular o R^2 nos dados de teste
    r2_test = lasso.score(X_test_scaled, y_test)
    
    # Armazenar os resultados
    lasso_results[alpha] = {'R^2 (treinamento)': r2_train, 'R^2 (teste)': r2_test}

# Imprimir os resultados
for alpha, result in lasso_results.items():
    print(f"Alpha = {alpha}: R^2 (treinamento) = {result['R^2 (treinamento)']}, R^2 (teste) = {result['R^2 (teste)']}")

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.858e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

Alpha = 0: R^2 (treinamento) = 0.2598159918234846, R^2 (teste) = 0.2690308673401247
Alpha = 0.001: R^2 (treinamento) = 0.2598159911623643, R^2 (teste) = 0.2690308653726393
Alpha = 0.005: R^2 (treinamento) = 0.25981598799705097, R^2 (teste) = 0.26903085701459406
Alpha = 0.01: R^2 (treinamento) = 0.25981598286823326, R^2 (teste) = 0.2690308454691146
Alpha = 0.05: R^2 (treinamento) = 0.2598158949365508, R^2 (teste) = 0.26903044034096146
Alpha = 0.1: R^2 (treinamento) = 0.2598156678057444, R^2 (teste) = 0.2690299069787111


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.105e+10, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


O melhor modelo seria aquele com o maior valor de R^2 nos dados de teste, pois indica que o modelo tem a melhor capacidade de explicar a variabilidade da variável dependente (renda). Neste caso, o R^2 é praticamente o mesmo para todos os valores de alpha, o que sugere que a regressão LASSO não está tendo um efeito significativo na performance do modelo.

In [133]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import patsy
import statsmodels.api as sm

# Carregar a base de dados
df = pd.read_csv('previsao_de_renda.csv')

# Remover as colunas 'Unnamed: 0', 'data_ref' e 'id_cliente'
df = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])

# Criar variáveis dummy
formula = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia - 1'
y, X = patsy.dmatrices(formula, df, return_type='dataframe')

# Separar em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Definir função para seleção de características stepwise
def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionado {best_feature} com p-value {best_pval}')
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Removido {worst_feature} com p-value {worst_pval}')
        if not changed:
            break
    return included

# Executar o modelo stepwise
selected_features = stepwise_selection(X_train, y_train['renda'])

# Ajustar o modelo final aos dados de treinamento
X_train_stepwise = X_train[selected_features]
model = LinearRegression()
model.fit(X_train_stepwise, y_train)

# Aplicar a seleção de características aos dados de teste
X_test_stepwise = X_test[selected_features]

# Calcular o R^2 na base de teste
r2_test = model.score(X_test_stepwise, y_test)
print(f"R^2 na base de teste: {r2_test}")

Adicionado sexo[M] com p-value 4.451346433934053e-205
Adicionado tipo_renda[T.Servidor público] com p-value 1.4969129039111874e-11
Adicionado tipo_renda[T.Pensionista] com p-value 1.090123161790028e-08
Adicionado educacao[T.Superior incompleto] com p-value 6.867388441695818e-06
Adicionado tipo_residencia[T.Com os pais] com p-value 0.00012096688830698076
Adicionado educacao[T.Superior completo] com p-value 0.006015710774399351
Adicionado sexo[F] com p-value 0.0011966903987365187
R^2 na base de teste: 0.09554670063774084


O melhor modelo seria aquele com o maior valor de R^2 nos dados de teste, pois indica que o modelo tem a melhor capacidade de explicar a variabilidade da variável dependente (renda). Neste caso, o R^2 eh muito baixo para tirar uma conclucao.

Achei o regularizacao ridge o melhor ja que deu o maior R^2

In [134]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

# Engenharia de características: Adicionar interações polinomiais de segundo grau
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Ajustar o modelo de regressão linear aos dados de treinamento
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

# Avaliar o R^2 na base de testes
r2_test_poly = model_poly.score(X_test_poly, y_test)
print(f"R^2 na base de teste com interações polinomiais: {r2_test_poly}")

# Seleção de características: Utilizar o modelo Lasso para selecionar características
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
selected_features_lasso = X_train.columns[lasso.coef_ != 0]

# Ajustar o modelo final aos dados de treinamento com as características selecionadas
X_train_selected_lasso = X_train[selected_features_lasso]
model_selected_lasso = LinearRegression()
model_selected_lasso.fit(X_train_selected_lasso, y_train)

# Aplicar a seleção de características aos dados de teste
X_test_selected_lasso = X_test[selected_features_lasso]

# Calcular o R^2 na base de teste
r2_test_selected_lasso = model_selected_lasso.score(X_test_selected_lasso, y_test)
print(f"R^2 na base de teste com seleção de características pelo Lasso: {r2_test_selected_lasso}")

R^2 na base de teste com interações polinomiais: -6.53164631030022e+16
R^2 na base de teste com seleção de características pelo Lasso: 0.0970731342188188


In [135]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# Ajustar uma árvore de regressão aos dados de treinamento
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train_stepwise, y_train)

# Fazer previsões na base de teste
y_pred_tree = tree_reg.predict(X_test_stepwise)

# Calcular o R^2 na base de teste
r2_test_tree = r2_score(y_test, y_pred_tree)
print(f"R^2 na base de teste com árvore de regressão: {r2_test_tree}")

R^2 na base de teste com árvore de regressão: 0.09669755295702775
